# 84와 85 합친 df 만들기

In [1]:
import pandas as pd
import numpy as np

origin_84 = pd.read_excel('./DATASET/fully_translated_df_with_hs.xlsx')
less_84 = pd.read_excel('./DATASET/84_only_one.xlsx')
none_84 = pd.read_excel('./DATASET/84_None.xlsx')
origin_85 = pd.read_excel('./DATASET/fully_translated_df_hs85.xlsx')
less_85 = pd.read_excel('./DATASET/85_less.xlsx')
none_85 = pd.read_excel('./DATASET/85_none.xlsx')

print(origin_84.columns, less_84.columns, none_84.columns)
print()
print(origin_85.columns, less_85.columns, none_85.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'hs code', 'product', 'translated_des',
       'translated_reason'],
      dtype='object') Index(['hs code', 'product', 'description', 'reason'], dtype='object') Index(['hs code', 'product ', 'description', 'reason'], dtype='object')

Index(['Unnamed: 0', 'hs code', 'translated_product_290', 'translated_des',
       'translated_reason'],
      dtype='object') Index(['hs code', 'product', 'description', 'reason'], dtype='object') Index(['hs code', 'product', 'description', 'reason'], dtype='object')


In [2]:
origin_84.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)
origin_85.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
origin_84.columns = ['hs code', 'product', 'description', 'reason']
origin_85.columns = ['hs code', 'product', 'description', 'reason']

In [4]:
df_84 = pd.concat([origin_84, less_84, none_84])
df_85 = pd.concat([origin_85, less_85, none_85])

print(len(df_84))
print(len(df_85))

3868
4835


In [6]:
df_84['product'] = df_84['product'].astype(str)

product84_des_list = []

for name, des in zip(df_84['product'], df_84['description']):
    product_des = name + des
    product84_des_list.append(product_des)

# product_des_list[0]
product84_des_df = pd.DataFrame({'hs code':df_84['hs code'], 'description':product84_des_list, 'reason':df_84['reason']})

In [7]:
df_85['product'] = df_85['product'].astype(str)

product85_des_list = []

for name, des in zip(df_85['product'], df_85['description']):
    product_des = name + des
    product85_des_list.append(product_des)

# product_des_list[0]
product85_des_df = pd.DataFrame({'hs code':df_85['hs code'], 'description':product85_des_list, 'reason':df_85['reason']})

In [14]:
product84_des_df['reason'].info()

<class 'pandas.core.series.Series'>
Index: 3868 entries, 0 to 17
Series name: reason
Non-Null Count  Dtype 
--------------  ----- 
3867 non-null   object
dtypes: object(1)
memory usage: 60.4+ KB


0     ㅇ The tariff rate No. 8428 is classified as "o...
1     -The tariff rate table 16 Lord 3, “The combine...
2     -The tariff rate table 8414 said, “Gastric pum...
3     ㅇ The tariff rate table No. 16, the No. 2, sta...
4     -Carcissization Table in the 16th parent No. 2...
                            ...                        
13    The applicable subheading for the oven/range h...
14    The applicable subheading for the grease fitti...
15    The applicable subheading for the dispense tip...
16    84.53 Machinery for preparing, tanning or work...
17    84.53 Machinery for preparing, tanning or work...
Name: reason, Length: 3868, dtype: object

In [17]:
import re
from sklearn.model_selection import train_test_split

x_84 = product84_des_df['description'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
y_84 = product84_des_df['hs code']
z_84 = product84_des_df['reason'] #.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

x_85 = product85_des_df['description'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
y_85 = product85_des_df['hs code']
z_85 = product85_des_df['reason'] #.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

x = pd.concat([x_84, x_85])
y = pd.concat([y_84, y_85])
z = pd.concat([z_84, z_85])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=10)

print(len(x_train))
print(len(x_test))

x_train = pd.concat([x_train, z])
y_train = pd.concat([y_train, y])

print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(x_test))

6092
2611
14795
14795
2611
2611


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import sent_tokenize
from nltk import word_tokenize
import nltk
import string
from nltk.stem import WordNetLemmatizer
import nltk


nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
# 특수문자
string = string.punctuation
# 불용어 + 특수문자 합친 리스트 생성
for stri in string:
    stopwords.append(stri)

def text_preprocessing(text):
    # 어근 추출 객체 생성
    lemmar = WordNetLemmatizer()

    # 텍스트를 소문자로 변환 후 문장으로 토큰화
    sentences = sent_tokenize(text.lower())

    for sentence in sentences:        
        # 단어로 토큰화
        tokens = word_tokenize(sentence)
        # 토큰화된 단어들의 어근 추출
        token_list = [lemmar.lemmatize(token) for token in tokens]

    return token_list

tfidf_vec = TfidfVectorizer(tokenizer=text_preprocessing, stop_words = stopwords)
tfidf_vec.fit(x_train.values.astype('U'))
x_train_tfidf_vec = tfidf_vec.transform(x_train.values.astype('U')) 

x_test_tfidf_vec = tfidf_vec.transform(x_test)
y_test = y_test.apply(pd.to_numeric, errors='coerce')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', '

In [19]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
svc = LinearSVC(C=1)
svc.fit(x_train_tfidf_vec,y_train)

y_hat = svc.predict(x_test_tfidf_vec)

print(f'정확도: {accuracy_score(y_test, y_hat):.3f}')
print(f'정밀도: {precision_score(y_test, y_hat, average="weighted"):.3f}')
print(f'재현율: {recall_score(y_test, y_hat, average="weighted"):.3f}')

정확도: 0.813
정밀도: 0.808
재현율: 0.813


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score

lr = LogisticRegression(solver='liblinear')
lr.fit(x_train_tfidf_vec,y_train)

y_hat = lr.predict(x_test_tfidf_vec)

print(f'정확도: {accuracy_score(y_test, y_hat):.3f}')
print(f'정밀도: {precision_score(y_test, y_hat, average="weighted"):.3f}')
print(f'재현율: {recall_score(y_test, y_hat, average="weighted"):.3f}')

정확도: 0.702
정밀도: 0.697
재현율: 0.702


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier


rmc =RandomForestClassifier(n_estimators=5000, max_depth=300)
rmc.fit(x_train_tfidf_vec,y_train)

y_hat = rmc.predict(x_test_tfidf_vec)

print(f'정확도: {accuracy_score(y_test, y_hat):.3f}')
print(f'정밀도: {precision_score(y_test, y_hat, average="weighted"):.3f}')
print(f'재현율: {recall_score(y_test, y_hat, average="weighted"):.3f}')

In [ ]:
test_df = pd.read_excel('./DATASET/testing.xlsx')
x_test = test_df['description']
